## Scripts for ISC analysis on Emotion EEG

### Basic setting

In [ ]:
%matplotlib inline
import os
import mne
from mne.viz import plot_topomap
from mne.time_frequency import psd_multitaper, psd_welch
from mne.stats import fdr_correction
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from jupyterthemes import jtplot
jtplot.style(theme='grade3') 
import seaborn as sns
from sklearn.metrics import pairwise_distances
from nltools.stats import matrix_permutation, fdr
from scipy.spatial.distance import pdist
from nltools.stats import isc
from neurora.stuff import permutation_corr
from Function import import_eeg, singch_psd, mean_df2_eeg, mean_df3_eeg

### Subject list
sub_ag = [1, 2] + list(range(4,12)) + [13,14,16,17] + list(range(19,24)) + [25,26,28,29,30,31] + list(range(33,43)) + [46] # 36
sub_ax = list(range(1,5)) + list(range(7,17)) + [19,20,21,22,23,25,26,28,29,30,31] + list(range(33,42)) + [43,44,45,46] # 38
sub_fe = list(range(1,13)) + [14,17,19,20,21,22,23] + [25,26,28,29,30,31] + list(range(33,41)) + [42,44,45,46] # 37
sub_hl = [1,2] + list(range(4,18)) + [19,20,21,22,23,25,26,28,29,30,31] + list(range(33,42)) + [43,44,46] # 39
sub_ha = list(range(1,18)) + [19,20,21,22,23,25,26,28,29,31] + list(range(33,47)) # 41

### Define data path

In [ ]:
# Define the pathectory
ag_path = 'F:/1_Emotion_Data/Data/EEG/Angry/'
ax_path = 'F:/1_Emotion_Data/Data/EEG/Anxiety/'
fe_path = 'F:/1_Emotion_Data/Data/EEG/Fear/'
hl_path = 'F:/1_Emotion_Data/Data/EEG/Helpless/'
ha_path = 'F:/1_Emotion_Data/Data/EEG/Happy/'
rest_path = 'F:/1_Emotion_Data/Data/EEG/Resting/'
results_path = 'F:/1_Emotion_Data/Results/0_ISC/EEG/'
plot_path = 'F:/1_Emotion_Data/Results/2_Plots/ISC/EEG/'

### Import emotion EEG datasets

In [ ]:
# Import angry meta data
ag_sub, ag1_meta = import_eeg('ag1', os.path.join(ag_path,'ag1/'))
ag_sub, ag2_meta = import_eeg('ag2', os.path.join(ag_path,'ag2/'))
ag_sub, ag3_meta = import_eeg('ag3', os.path.join(ag_path,'ag3/'))

In [ ]:
# Import anxiety meta data
ax_sub, ax1_meta = import_eeg('ax1', os.path.join(ax_path,'ax1/'))
ax_sub, ax2_meta = import_eeg('ax2', os.path.join(ax_path,'ax2/'))

In [ ]:
# Import fear meta data
fe_sub, fe1_meta = import_eeg('fe1', os.path.join(fe_path,'fe1/'))
fe_sub, fe2_meta = import_eeg('fe2', os.path.join(fe_path,'fe2/'))
fe_sub, fe3_meta = import_eeg('fe3', os.path.join(fe_path,'fe3/'))

In [ ]:
# Import helpless meta data
hl_sub, hl2_meta = import_eeg('hl2', os.path.join(hl_path,'hl2/'))
hl_sub, hl3_meta = import_eeg('hl3', os.path.join(hl_path,'hl3/'))

In [ ]:
# Import happy meta data
ha_sub, ha1_meta = import_eeg('ha1', os.path.join(ha_path,'ha1/'))
ha_sub, ha3_meta = import_eeg('ha3', os.path.join(ha_path,'ha3/'))

### Obtain channel index and topo information

In [ ]:
# Obtain channel & montage information for topo-plot
topo_info = ag1_meta[1].info
# Obtain the channel names as a list
ch_names = ag1_meta[1].ch_names
# Crate a index list for channels
ch_idx = list(range(63))
# Combine the channels and index and convert to a dict
ch_names = dict(zip(ch_names, ch_idx))
print(ch_names['Fpz'])

### Section 1: ISC of *Angry* EEG

Angry annotations:
* ag1: 220 - 240
* ag2: 125 - 145
* ag3: 65 - 85

In [ ]:
ch_idx = list(range(63))
r_topo = np.zeros((63,5))
simi = np.zeros((630,63,5))

psd1 = np.zeros((63,5,36,21))
for j in range(219, 240):
    ag = np.zeros((63,50,36))
    for sub in sub_ag:
        epoch = ag1_meta[sub][j]
        psds, freqs = psd_multitaper(epoch,fmin=1,fmax=50)
        psds = np.squeeze(10* np.log10(10**12*psds))
        idx = sub_ag.index(sub)
        ag[:,:,idx] = psds
    
    idx = range(219,240).index(j)
    psd1[:,0,:, idx] = ag[:,0:4,:].mean(1)
    psd1[:,1,:, idx] = ag[:,3:8,:].mean(1)
    psd1[:,2,:, idx] = ag[:,7:12,:].mean(1)
    psd1[:,3,:, idx] = ag[:,11:30,:].mean(1)
    psd1[:,4,:, idx] = ag[:,29:45,:].mean(1)

psd2 = np.zeros((63,5,36,21))
for j in range(124, 145):
    ag = np.zeros((63,50,36))
    for sub in sub_ag:
        epoch = ag2_meta[sub][j]
        psds, freqs = psd_multitaper(epoch,fmin=1,fmax=50)
        psds = np.squeeze(10* np.log10(10**12*psds))
        idx = sub_ag.index(sub)
        ag[:,:,idx] = psds
    
    idx = range(124,145).index(j)
    psd2[:,0,:, idx] = ag[:,0:4,:].mean(1)
    psd2[:,1,:, idx] = ag[:,3:8,:].mean(1)
    psd2[:,2,:, idx] = ag[:,7:12,:].mean(1)
    psd2[:,3,:, idx] = ag[:,11:30,:].mean(1)
    psd2[:,4,:, idx] = ag[:,29:45,:].mean(1)

psd3 = np.zeros((63,5,36,21))
for j in range(64, 85):
    ag = np.zeros((63,50,36))
    for sub in sub_ag:
        epoch = ag3_meta[sub][j]
        psds, freqs = psd_multitaper(epoch,fmin=1,fmax=50)
        psds = np.squeeze(10* np.log10(10**12*psds))
        idx = sub_ag.index(sub)
        ag[:,:,idx] = psds
    
    idx = range(64,85).index(j)
    psd3[:,0,:, idx] = ag[:,0:4,:].mean(1)
    psd3[:,1,:, idx] = ag[:,3:8,:].mean(1)
    psd3[:,2,:, idx] = ag[:,7:12,:].mean(1)
    psd3[:,3,:, idx] = ag[:,11:30,:].mean(1)
    psd3[:,4,:, idx] = ag[:,29:45,:].mean(1)
    
psd = np.mean(np.stack((psd1, psd2, psd3),axis=4),axis=4)

for ch in ch_idx:
    simi_delta = 1-pdist(psd[ch,0,:,:], metric='correlation')
    simi[:,ch,0] = simi_delta
    r_topo[ch,0] = simi_delta.mean()

    simi_theta = 1-pdist(psd[ch,1,:,:], metric='correlation')
    simi[:,ch,1] = simi_theta
    r_topo[ch,1] = simi_theta.mean()

    simi_alpha = 1-pdist(psd[ch,2,:,:], metric='correlation')
    simi[:,ch,2] = simi_alpha
    r_topo[ch,2] = simi_alpha.mean()

    simi_beta = 1-pdist(psd[ch,3,:,:], metric='correlation')
    simi[:,ch,3] = simi_beta
    r_topo[ch,3] = simi_beta.mean()

    simi_gamma = 1-pdist(psd[ch,4,:,:], metric='correlation')
    simi[:,ch,4] = simi_gamma
    r_topo[ch,4] = simi_gamma.mean()

isc = {'r_topo':r_topo, 'matrix':simi}
np.save(os.path.join(results_path, 'ag_isc_matrix.npy'), isc) 

### Angry ISC topoplot

In [ ]:
isc = np.load(os.path.join(results_path, 'ag_isc_matrix.npy'), allow_pickle=True).item()
r_topo = isc['r_topo']

fig,(ax1, ax2, ax3, ax4, ax5) = plt.subplots(ncols=5, figsize=(25,15))
im,_ = plot_topomap(r_topo[:,0], topo_info, axes=ax1, show=False, vmin=0, vmax=0.15, cmap='Reds',sphere=0.13)
im,_ = plot_topomap(r_topo[:,1], topo_info, axes=ax2, show=False, vmin=0, vmax=0.15,  cmap='Reds',sphere=0.13)    
im,_ = plot_topomap(r_topo[:,2], topo_info, axes=ax3, show=False, vmin=0, vmax=0.15,  cmap='Reds',sphere=0.13)    
im,_ = plot_topomap(r_topo[:,3], topo_info, axes=ax4, show=False, vmin=0, vmax=0.15, cmap='Reds',sphere=0.13)    
im,_ = plot_topomap(r_topo[:,4], topo_info, axes=ax5, show=False, vmin=0, vmax=0.15, cmap='Reds',sphere=0.13)  
ax1.set_title('Delta',fontsize=20) 
ax2.set_title('Theta',fontsize=20) 
ax3.set_title('Alpha',fontsize=20) 
ax4.set_title('Beta',fontsize=20) 
ax5.set_title('Gamma',fontsize=20) 
ax_x_start, ax_x_width, ax_y_start, ax_y_height  = 0.95, 0.015, 0.4, 0.25
cbar_ax = fig.add_axes([ax_x_start, ax_y_start, ax_x_width, ax_y_height])
clb = fig.colorbar(im, cax=cbar_ax)
fig.suptitle('Angry ISC', x=0.5, y=0.75, fontsize=30, fontweight='bold')
plt.show()
# plt.savefig('F:/1_Emotion_Data/Results/2_Plots/ISC/EEG/ag_psd_isc.png',bbox_inches='tight',dpi=600,pad_inches=0.1)
# plt.close()

### Section 2: ISC for *Anxiety* EEG

Anxiety annotation:
* ax1: 135 - 155
* ax2: 135 - 155

In [ ]:
ch_idx = list(range(63))
r_topo = np.zeros((63,5))
simi = np.zeros((703,63,5))

psd1 = np.zeros((63,5,38,21))
for j in range(134, 155):
    ax = np.zeros((63,50,38))
    for sub in sub_ax:
        epoch = ax1_meta[sub][j]
        psds, freqs = psd_multitaper(epoch,fmin=1,fmax=50)
        psds = np.squeeze(10* np.log10(10**12*psds))
        idx = sub_ax.index(sub)
        ax[:,:,idx] = psds
    
    idx = range(134,155).index(j)
    psd1[:,0,:, idx] = ax[:,0:4,:].mean(1)
    psd1[:,1,:, idx] = ax[:,3:8,:].mean(1)
    psd1[:,2,:, idx] = ax[:,7:12,:].mean(1)
    psd1[:,3,:, idx] = ax[:,11:30,:].mean(1)
    psd1[:,4,:, idx] = ax[:,29:45,:].mean(1)

psd2 = np.zeros((63,5,38,21))
for j in range(134, 155):
    ax = np.zeros((63,50,38))
    for sub in sub_ax:
        epoch = ax2_meta[sub][j]
        psds, freqs = psd_multitaper(epoch,fmin=1,fmax=50)
        psds = np.squeeze(10* np.log10(10**12*psds))
        idx = sub_ax.index(sub)
        ax[:,:,idx] = psds
    
    idx = range(134,155).index(j)
    psd2[:,0,:, idx] = ax[:,0:4,:].mean(1)
    psd2[:,1,:, idx] = ax[:,3:8,:].mean(1)
    psd2[:,2,:, idx] = ax[:,7:12,:].mean(1)
    psd2[:,3,:, idx] = ax[:,11:30,:].mean(1)
    psd2[:,4,:, idx] = ax[:,29:45,:].mean(1)

    
psd = np.mean(np.stack((psd1, psd2),axis=4),axis=4)

for ch in ch_idx:
    simi_delta = 1-pdist(psd[ch,0,:,:], metric='correlation')
    simi[:,ch,0] = simi_delta
    r_topo[ch,0] = simi_delta.mean()

    simi_theta = 1-pdist(psd[ch,1,:,:], metric='correlation')
    simi[:,ch,1] = simi_theta
    r_topo[ch,1] = simi_theta.mean()

    simi_alpha = 1-pdist(psd[ch,2,:,:], metric='correlation')
    simi[:,ch,2] = simi_alpha
    r_topo[ch,2] = simi_alpha.mean()

    simi_beta = 1-pdist(psd[ch,3,:,:], metric='correlation')
    simi[:,ch,3] = simi_beta
    r_topo[ch,3] = simi_beta.mean()

    simi_gamma = 1-pdist(psd[ch,4,:,:], metric='correlation')
    simi[:,ch,4] = simi_gamma
    r_topo[ch,4] = simi_gamma.mean()

isc = {'r_topo':r_topo, 'matrix':simi}
np.save(os.path.join(results_path, 'ax_isc_matrix.npy'), isc) 

### Anxiety ISC topoplots

In [ ]:
isc = np.load(os.path.join(results_path, 'ax_isc_matrix.npy'), allow_pickle=True).item()
r_topo = isc['r_topo']

fig,(ax1, ax2, ax3, ax4, ax5) = plt.subplots(ncols=5, figsize=(25,15))
im,_ = plot_topomap(r_topo[:,0], topo_info, axes=ax1, show=False, vmin=0, vmax=0.15, cmap='Reds',sphere=0.13)
im,_ = plot_topomap(r_topo[:,1], topo_info, axes=ax2, show=False, vmin=0, vmax=0.15,  cmap='Reds',sphere=0.13)    
im,_ = plot_topomap(r_topo[:,2], topo_info, axes=ax3, show=False, vmin=0, vmax=0.15,  cmap='Reds',sphere=0.13)    
im,_ = plot_topomap(r_topo[:,3], topo_info, axes=ax4, show=False, vmin=0, vmax=0.15, cmap='Reds',sphere=0.13)    
im,_ = plot_topomap(r_topo[:,4], topo_info, axes=ax5, show=False, vmin=0, vmax=0.15, cmap='Reds',sphere=0.13)  
# ax1.set_title('Delta',fontsize=20) 
# ax2.set_title('Theta',fontsize=20) 
# ax3.set_title('Alpha',fontsize=20) 
# ax4.set_title('Beta',fontsize=20) 
# ax5.set_title('Gamma',fontsize=20) 
ax_x_start, ax_x_width, ax_y_start, ax_y_height  = 0.95, 0.015, 0.4, 0.25
cbar_ax = fig.add_axes([ax_x_start, ax_y_start, ax_x_width, ax_y_height])
clb = fig.colorbar(im, cax=cbar_ax)
#fig.suptitle('Anxiety ISC', x=0.5, y=0.75, fontsize=30, fontweight='bold')
plt.show()
# plt.savefig('F:/1_Emotion_Data/Results/2_Plots/ISC/EEG/ax_psd_isc.png',bbox_inches='tight',dpi=600,pad_inches=0.1)
# plt.close()

### Section 3: ISC for *Fear* EEG

Fear annotations
* fe1: 110- 130
* fe2: 30 - 50
* fe3: 45 - 65

In [ ]:
ch_idx = list(range(63))
r_topo = np.zeros((63,5))
simi = np.zeros((666,63,5))

psd1 = np.zeros((63,5,37,21))
for j in range(109, 130):
    fe = np.zeros((63,50,37))
    for sub in sub_fe:
        epoch = fe1_meta[sub][j]
        psds, freqs = psd_multitaper(epoch,fmin=1,fmax=50)
        psds = np.squeeze(10* np.log10(10**12*psds))
        idx = sub_fe.index(sub)
        fe[:,:,idx] = psds
    
    idx = range(109,130).index(j)
    psd1[:,0,:, idx] = fe[:,0:4,:].mean(1)
    psd1[:,1,:, idx] = fe[:,3:8,:].mean(1)
    psd1[:,2,:, idx] = fe[:,7:12,:].mean(1)
    psd1[:,3,:, idx] = fe[:,11:30,:].mean(1)
    psd1[:,4,:, idx] = fe[:,29:45,:].mean(1)

psd2 = np.zeros((63,5,37,21))
for j in range(29, 50):
    fe = np.zeros((63,50,37))
    for sub in sub_fe:
        epoch = fe2_meta[sub][j]
        psds, freqs = psd_multitaper(epoch,fmin=1,fmax=50)
        psds = np.squeeze(10* np.log10(10**12*psds))
        idx = sub_fe.index(sub)
        fe[:,:,idx] = psds
    
    idx = range(29,50).index(j)
    psd2[:,0,:, idx] = fe[:,0:4,:].mean(1)
    psd2[:,1,:, idx] = fe[:,3:8,:].mean(1)
    psd2[:,2,:, idx] = fe[:,7:12,:].mean(1)
    psd2[:,3,:, idx] = fe[:,11:30,:].mean(1)
    psd2[:,4,:, idx] = fe[:,29:45,:].mean(1)

psd3 = np.zeros((63,5,37,21))
for j in range(44, 65):
    fe = np.zeros((63,50,37))
    for sub in sub_fe:
        epoch = fe3_meta[sub][j]
        psds, freqs = psd_multitaper(epoch,fmin=1,fmax=50)
        psds = np.squeeze(10* np.log10(10**12*psds))
        idx = sub_fe.index(sub)
        fe[:,:,idx] = psds
    
    idx = range(44,65).index(j)
    psd3[:,0,:, idx] = fe[:,0:4,:].mean(1)
    psd3[:,1,:, idx] = fe[:,3:8,:].mean(1)
    psd3[:,2,:, idx] = fe[:,7:12,:].mean(1)
    psd3[:,3,:, idx] = fe[:,11:30,:].mean(1)
    psd3[:,4,:, idx] = fe[:,29:45,:].mean(1)
    
psd = np.mean(np.stack((psd1, psd2, psd3),axis=4),axis=4)

for ch in ch_idx:
    simi_delta = 1-pdist(psd[ch,0,:,:], metric='correlation')
    simi[:,ch,0] = simi_delta
    r_topo[ch,0] = simi_delta.mean()

    simi_theta = 1-pdist(psd[ch,1,:,:], metric='correlation')
    simi[:,ch,1] = simi_theta
    r_topo[ch,1] = simi_theta.mean()

    simi_alpha = 1-pdist(psd[ch,2,:,:], metric='correlation')
    simi[:,ch,2] = simi_alpha
    r_topo[ch,2] = simi_alpha.mean()

    simi_beta = 1-pdist(psd[ch,3,:,:], metric='correlation')
    simi[:,ch,3] = simi_beta
    r_topo[ch,3] = simi_beta.mean()

    simi_gamma = 1-pdist(psd[ch,4,:,:], metric='correlation')
    simi[:,ch,4] = simi_gamma
    r_topo[ch,4] = simi_gamma.mean()

isc = {'r_topo':r_topo, 'matrix':simi}
np.save(os.path.join(results_path, 'fe_isc_matrix.npy'), isc) 

### Fear ISC topoplots

In [ ]:
isc = np.load(os.path.join(results_path, 'fe_isc_matrix.npy'), allow_pickle=True).item()
r_topo = isc['r_topo']

fig,(ax1, ax2, ax3, ax4, ax5) = plt.subplots(ncols=5, figsize=(25,15))
im,_ = plot_topomap(r_topo[:,0], topo_info, axes=ax1, show=False, vmin=0, vmax=0.15, cmap='Reds',sphere=0.13)
im,_ = plot_topomap(r_topo[:,1], topo_info, axes=ax2, show=False, vmin=0, vmax=0.15,  cmap='Reds',sphere=0.13)    
im,_ = plot_topomap(r_topo[:,2], topo_info, axes=ax3, show=False, vmin=0, vmax=0.15,  cmap='Reds',sphere=0.13)    
im,_ = plot_topomap(r_topo[:,3], topo_info, axes=ax4, show=False, vmin=0, vmax=0.15, cmap='Reds',sphere=0.13)    
im,_ = plot_topomap(r_topo[:,4], topo_info, axes=ax5, show=False, vmin=0, vmax=0.15, cmap='Reds',sphere=0.13)  
# ax1.set_title('Delta',fontsize=20) 
# ax2.set_title('Theta',fontsize=20) 
# ax3.set_title('Alpha',fontsize=20) 
# ax4.set_title('Beta',fontsize=20) 
# ax5.set_title('Gamma',fontsize=20) 
ax_x_start, ax_x_width, ax_y_start, ax_y_height  = 0.95, 0.015, 0.4, 0.25
cbar_ax = fig.add_axes([ax_x_start, ax_y_start, ax_x_width, ax_y_height])
clb = fig.colorbar(im, cax=cbar_ax)
#fig.suptitle('Fear ISC', x=0.5, y=0.75, fontsize=30, fontweight='bold')
plt.show()
# plt.savefig('F:/1_Emotion_Data/Results/2_Plots/ISC/EEG/fe_psd_isc.png',bbox_inches='tight',dpi=600,pad_inches=0.1)
# plt.close()

### Section 4: ISC for *Helpless* EEG

Helpless annotations
* hl2: 115 - 135
* hl3: 50 - 70

In [ ]:
ch_idx = list(range(63))

r_topo = np.zeros((63,5))
simi = np.zeros((741,63,5))

psd2 = np.zeros((63,5,39,21))
for j in range(114, 135):
    hl = np.zeros((63,50,39))
    for sub in sub_hl:
        epoch = hl2_meta[sub][j]
        psds, freqs = psd_multitaper(epoch,fmin=1,fmax=50)
        psds = np.squeeze(10* np.log10(10**12*psds))
        idx = sub_hl.index(sub)
        hl[:,:,idx] = psds
    
    idx = range(114,135).index(j)
    psd2[:,0,:, idx] = hl[:,0:4,:].mean(1)
    psd2[:,1,:, idx] = hl[:,3:8,:].mean(1)
    psd2[:,2,:, idx] = hl[:,7:12,:].mean(1)
    psd2[:,3,:, idx] = hl[:,11:30,:].mean(1)
    psd2[:,4,:, idx] = hl[:,29:45,:].mean(1)

psd3 = np.zeros((63,5,39,21))
for j in range(49, 70):
    hl = np.zeros((63,50,39))
    for sub in sub_hl:
        epoch = hl3_meta[sub][j]
        psds, freqs = psd_multitaper(epoch,fmin=1,fmax=50)
        psds = np.squeeze(10* np.log10(10**12*psds))
        idx = sub_hl.index(sub)
        hl[:,:,idx] = psds
    
    idx = range(49,70).index(j)
    psd3[:,0,:, idx] = hl[:,0:4,:].mean(1)
    psd3[:,1,:, idx] = hl[:,3:8,:].mean(1)
    psd3[:,2,:, idx] = hl[:,7:12,:].mean(1)
    psd3[:,3,:, idx] = hl[:,11:30,:].mean(1)
    psd3[:,4,:, idx] = hl[:,29:45,:].mean(1)
    
psd = np.mean(np.stack((psd2, psd3),axis=4),axis=4)

for ch in ch_idx:
    simi_delta = 1-pdist(psd[ch,0,:,:], metric='correlation')
    simi[:,ch,0] = simi_delta
    r_topo[ch,0] = simi_delta.mean()

    simi_theta = 1-pdist(psd[ch,1,:,:], metric='correlation')
    simi[:,ch,1] = simi_theta
    r_topo[ch,1] = simi_theta.mean()

    simi_alpha = 1-pdist(psd[ch,2,:,:], metric='correlation')
    simi[:,ch,2] = simi_alpha
    r_topo[ch,2] = simi_alpha.mean()

    simi_beta = 1-pdist(psd[ch,3,:,:], metric='correlation')
    simi[:,ch,3] = simi_beta
    r_topo[ch,3] = simi_beta.mean()

    simi_gamma = 1-pdist(psd[ch,4,:,:], metric='correlation')
    simi[:,ch,4] = simi_gamma
    r_topo[ch,4] = simi_gamma.mean()

isc = {'r_topo':r_topo, 'matrix':simi}
np.save(os.path.join(results_path, 'hl_isc_matrix.npy'), isc) 

### Helpless ISC topoplots

In [ ]:
isc = np.load(os.path.join(results_path, 'hl_isc_matrix.npy'), allow_pickle=True).item()
r_topo = isc['r_topo']

fig,(ax1, ax2, ax3, ax4, ax5) = plt.subplots(ncols=5, figsize=(25,15))
im,_ = plot_topomap(r_topo[:,0], topo_info, axes=ax1, show=False, vmin=0, vmax=0.15, cmap='Reds',sphere=0.13)
im,_ = plot_topomap(r_topo[:,1], topo_info, axes=ax2, show=False, vmin=0, vmax=0.15,  cmap='Reds',sphere=0.13)    
im,_ = plot_topomap(r_topo[:,2], topo_info, axes=ax3, show=False, vmin=0, vmax=0.15,  cmap='Reds',sphere=0.13)    
im,_ = plot_topomap(r_topo[:,3], topo_info, axes=ax4, show=False, vmin=0, vmax=0.15, cmap='Reds',sphere=0.13)    
im,_ = plot_topomap(r_topo[:,4], topo_info, axes=ax5, show=False, vmin=0, vmax=0.15, cmap='Reds',sphere=0.13)  
# ax1.set_title('Delta',fontsize=20) 
# ax2.set_title('Theta',fontsize=20) 
# ax3.set_title('Alpha',fontsize=20) 
# ax4.set_title('Beta',fontsize=20) 
# ax5.set_title('Gamma',fontsize=20) 
ax_x_start, ax_x_width, ax_y_start, ax_y_height  = 0.95, 0.015, 0.4, 0.25
cbar_ax = fig.add_axes([ax_x_start, ax_y_start, ax_x_width, ax_y_height])
clb = fig.colorbar(im, cax=cbar_ax)
#fig.suptitle('Helpless ISC', x=0.5, y=0.75, fontsize=30, fontweight='bold')
plt.show()
# plt.savefig('F:/1_Emotion_Data/Results/2_Plots/ISC/EEG/hl_psd_isc.png',bbox_inches='tight',dpi=600,pad_inches=0.1)
# plt.close()

### Section 5: ISC for *happy* EEG

 Happy dataset annotations:
* ha1: 180 - 200
* ha3: 5 - 25

In [ ]:
ch_idx = list(range(63))

r_topo = np.zeros((63,5))
simi = np.zeros((820,63,5))

psd1 = np.zeros((63,5,41,21))
for j in range(179, 200):
    ha = np.zeros((63,50,41))
    for sub in sub_ha:
        epoch = ha1_meta[sub][j]
        psds, freqs = psd_multitaper(epoch,fmin=1,fmax=50)
        psds = np.squeeze(10* np.log10(10**12*psds))
        idx = sub_ha.index(sub)
        ha[:,:,idx] = psds
    
    idx = range(179,200).index(j)
    psd1[:,0,:, idx] = ha[:,0:4,:].mean(1)
    psd1[:,1,:, idx] = ha[:,3:8,:].mean(1)
    psd1[:,2,:, idx] = ha[:,7:12,:].mean(1)
    psd1[:,3,:, idx] = ha[:,11:30,:].mean(1)
    psd1[:,4,:, idx] = ha[:,29:45,:].mean(1)

psd3 = np.zeros((63,5,41,21))
for j in range(4, 25):
    ha = np.zeros((63,50,41))
    for sub in sub_ha:
        epoch = ha3_meta[sub][j]
        psds, freqs = psd_multitaper(epoch,fmin=1,fmax=50)
        psds = np.squeeze(10* np.log10(10**12*psds))
        idx = sub_ha.index(sub)
        ha[:,:,idx] = psds
    
    idx = range(4,25).index(j)
    psd3[:,0,:, idx] = ha[:,0:4,:].mean(1)
    psd3[:,1,:, idx] = ha[:,3:8,:].mean(1)
    psd3[:,2,:, idx] = ha[:,7:12,:].mean(1)
    psd3[:,3,:, idx] = ha[:,11:30,:].mean(1)
    psd3[:,4,:, idx] = ha[:,29:45,:].mean(1)
    
psd = np.mean(np.stack((psd1, psd3),axis=4),axis=4)

for ch in ch_idx:
    simi_delta = 1-pdist(psd[ch,0,:,:], metric='correlation')
    simi[:,ch,0] = simi_delta
    r_topo[ch,0] = simi_delta.mean()

    simi_theta = 1-pdist(psd[ch,1,:,:], metric='correlation')
    simi[:,ch,1] = simi_theta
    r_topo[ch,1] = simi_theta.mean()

    simi_alpha = 1-pdist(psd[ch,2,:,:], metric='correlation')
    simi[:,ch,2] = simi_alpha
    r_topo[ch,2] = simi_alpha.mean()

    simi_beta = 1-pdist(psd[ch,3,:,:], metric='correlation')
    simi[:,ch,3] = simi_beta
    r_topo[ch,3] = simi_beta.mean()

    simi_gamma = 1-pdist(psd[ch,4,:,:], metric='correlation')
    simi[:,ch,4] = simi_gamma
    r_topo[ch,4] = simi_gamma.mean()

isc = {'r_topo':r_topo, 'matrix':simi}
np.save(os.path.join(results_path, 'ha_isc_matrix.npy'), isc) 

### Happy ISC topoplots

In [ ]:
isc = np.load(os.path.join(results_path, 'ha_isc_matrix.npy'), allow_pickle=True).item()
r_topo = isc['r_topo']

fig,(ax1, ax2, ax3, ax4, ax5) = plt.subplots(ncols=5, figsize=(25,15))
im,_ = plot_topomap(r_topo[:,0], topo_info, axes=ax1, show=False, vmin=0, vmax=0.15, cmap='Reds',sphere=0.13)
im,_ = plot_topomap(r_topo[:,1], topo_info, axes=ax2, show=False, vmin=0, vmax=0.15,  cmap='Reds',sphere=0.13)    
im,_ = plot_topomap(r_topo[:,2], topo_info, axes=ax3, show=False, vmin=0, vmax=0.15,  cmap='Reds',sphere=0.13)    
im,_ = plot_topomap(r_topo[:,3], topo_info, axes=ax4, show=False, vmin=0, vmax=0.15, cmap='Reds',sphere=0.13)    
im,_ = plot_topomap(r_topo[:,4], topo_info, axes=ax5, show=False, vmin=0, vmax=0.15, cmap='Reds',sphere=0.13)  
# ax1.set_title('Delta',fontsize=20) 
# ax2.set_title('Theta',fontsize=20) 
# ax3.set_title('Alpha',fontsize=20) 
# ax4.set_title('Beta',fontsize=20) 
# ax5.set_title('Gamma',fontsize=20) 
ax_x_start, ax_x_width, ax_y_start, ax_y_height  = 0.95, 0.015, 0.4, 0.25
cbar_ax = fig.add_axes([ax_x_start, ax_y_start, ax_x_width, ax_y_height])
clb = fig.colorbar(im, cax=cbar_ax)
# fig.suptitle('Happy ISC', x=0.5, y=0.75, fontsize=30, fontweight='bold')
plt.show()
# plt.savefig('F:/1_Emotion_Data/Results/2_Plots/ISC/EEG/ha_psd_isc.png',bbox_inches='tight',dpi=600,pad_inches=0.1)
# plt.close()